<a href="https://colab.research.google.com/github/BachVy/TRI-TUE-NHAN-TAO/blob/main/BaiTHTuan02_AStar_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile points.py
from math import sqrt
from typing import List
import random

class Point:
    def __init__(self, x: float, y: float, index: int):
        self.x = x
        self.y = y
        self.index = index

    def __repr__(self):
        return f"Point({self.index}: ({self.x:.2f}, {self.y:.2f}))"

def generate_random_points(n: int) -> List[Point]:
    """Tạo n điểm ngẫu nhiên: Điểm 0 là kho (0,0), các điểm khác random [0,5]x[0,5]"""
    if n < 1:
        raise ValueError("Số điểm n phải >= 1")
    points = [Point(0.0, 0.0, 0)]
    for i in range(1, n):
        x = random.uniform(0, 5)
        y = random.uniform(0, 5)
        points.append(Point(x, y, i))
    print(f"Đã tạo {n} điểm ngẫu nhiên: {points}")
    return points

def calculate_distance(p1: Point, p2: Point) -> float:
    dx = p1.x - p2.x
    dy = p1.y - p2.y
    return sqrt(dx*dx + dy*dy)

def get_distance_matrix(points: List[Point]) -> List[List[float]]:
    n = len(points)
    dist = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            dist[i][j] = calculate_distance(points[i], points[j])
    return dist

Writing points.py


In [2]:
%%writefile graph.py
from typing import List
from points import Point, get_distance_matrix

class TSPGraph:
    def __init__(self, points: List[Point]):
        self.points = points
        self.n = len(points)
        self.dist = get_distance_matrix(points)
        self.start_node = 0

    def heuristic(self, current: int, visited: int) -> float:
        """Heuristic admissible: min_dist * (remaining / 2)"""
        unvisited = [i for i in range(self.n) if not (visited & (1 << i))]
        if not unvisited:
            return 0.0
        min_dist = min(self.dist[current][j] for j in unvisited)
        remaining = len(unvisited)
        return min_dist * (remaining / 2.0)

    def is_goal(self, visited: int) -> bool:
        return visited == (1 << self.n) - 1

    def get_unvisited_neighbors(self, current: int, visited: int) -> List[int]:
        return [i for i in range(self.n) if not (visited & (1 << i))]

Writing graph.py


In [3]:
%%writefile solver.py
import heapq
from typing import List, Tuple, Dict
from graph import TSPGraph

class AStarSolver:
    def __init__(self, graph: TSPGraph):
        self.graph = graph
        self.visited_states: Dict[Tuple[int, int], float] = {}

    def calc_f(self, state: Tuple[int, int, float, List[int]]) -> float:
        current, visited, g, _ = state
        h = self.graph.heuristic(current, visited)
        return g + h

    def solve_with_visual(self, max_steps: int = 20) -> Tuple[List[int], float, List[Tuple[List[int], List[Tuple[int, int, float, List[int]]]]]]:
        """Solve A* với print steps chi tiết và trả steps cho visual"""
        start_state = (self.graph.start_node, 1 << self.graph.start_node, 0.0, [self.graph.start_node])
        start_h = self.graph.heuristic(self.graph.start_node, 1 << self.graph.start_node)
        start_f = self.calc_f(start_state)
        pq = [(-start_f, start_state)]
        heapq.heapify(pq)

        steps = []
        step = 0
        print(f"Step {step}: Khởi tạo - Current: {self.graph.start_node}, Path: [{self.graph.start_node}], g: {0:.2f}, h: {start_h:.2f}, f: {start_f:.2f}")
        steps.append(([self.graph.start_node], [start_state]))
        step += 1

        while pq and step <= max_steps:
            _, state = heapq.heappop(pq)
            current, visited, g, path = state
            state_key = (current, visited)
            if state_key in self.visited_states and self.visited_states[state_key] <= g:
                continue
            self.visited_states[state_key] = g

            print(f"\nStep {step}: Pop nút - Current: {current}, Visited: {bin(visited)[2:]} (điểm: {path}), g: {g:.2f}")

            if self.graph.is_goal(visited):
                h = 0
                f = g + h
                print(f"Step {step}: Tìm thấy goal! Path: {path}, Total g: {g:.2f}, h: {h:.2f}, f: {f:.2f}")
                steps.append((path, []))
                return path, g, steps

            # Mở rộng với print
            expansions = []
            for next_city in self.graph.get_unvisited_neighbors(current, visited):
                new_visited = visited | (1 << next_city)
                new_g = g + self.graph.dist[current][next_city]
                new_path = path + [next_city]
                h_new = self.graph.heuristic(next_city, new_visited)
                f_new = new_g + h_new
                expansions.append((f_new, next_city, new_g, h_new, new_path, new_visited))
                print(f"  Mở rộng đến {next_city}: g_new={new_g:.2f}, h_new={h_new:.2f}, f_new={f_new:.2f}")

            # Push vào queue
            for _, next_city, new_g, _, new_path, new_visited in sorted(expansions):
                new_state = (next_city, new_visited, new_g, new_path)
                heapq.heappush(pq, (-self.calc_f(new_state), new_state))

            # Lấy queue sample
            queue_sample = self._get_queue_sample(pq)
            print(f"  Queue sau mở rộng (top 3 f): {[round(self.calc_f(s), 2) for s in queue_sample]}")
            steps.append((path, queue_sample))
            step += 1

        print("Không tìm thấy goal trong max_steps.")
        return [], 0.0, steps

    def _get_queue_sample(self, pq: List[Tuple[float, Tuple]]) -> List[Tuple[int, int, float, List[int]]]:
        temp_pq = []
        sample = []
        while len(temp_pq) < 3 and pq:
            neg_f, state = heapq.heappop(pq)
            temp_pq.append((neg_f, state))
            sample.append(state)
        for neg_f, state in reversed(temp_pq):
            heapq.heappush(pq, (neg_f, state))
        return sample

Writing solver.py


In [4]:
%%writefile visualizer.py
import matplotlib.pyplot as plt
import imageio
import os
import shutil
from typing import List, Tuple
from points import Point

class Visualizer:
    def __init__(self, points: List[Point]):
        self.points = points
        self.results_dir = 'results'
        self.frames_dir = 'frames'

    def plot_step(self, step_num: int, current_path: List[int], queue_items: List[Tuple], title: str):
        """Vẽ và lưu PNG cho step vào frames_dir (tạm, không vào results)"""
        if not os.path.exists(self.frames_dir):
            os.makedirs(self.frames_dir)
        plt.figure(figsize=(8, 6))
        # Vẽ điểm
        for i, point in enumerate(self.points):
            color = 'red' if i in current_path else 'blue'
            plt.scatter(point.x, point.y, c=color, s=1000)
            plt.text(point.x, point.y, str(i), fontsize=12, ha='center')
        # Vẽ path
        if len(current_path) > 1:
            for i in range(len(current_path)-1):
                p1 = self.points[current_path[i]]
                p2 = self.points[current_path[i+1]]
                plt.arrow(p1.x, p1.y, p2.x - p1.x, p2.y - p1.y, head_width=0.1, head_length=0.1, fc='red', ec='red')
        # Queue text
        if queue_items:
            queue_text = "Queue top 3 (path, f):\n" + "\n".join(
                f"  {idx+1}: Path={str(state[3])}, f={self._calc_f(state):.2f}" for idx, state in enumerate(queue_items[:3])
            )
            plt.text(0.02, 0.98, queue_text, transform=plt.gca().transAxes, verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='wheat'), fontsize=8)
        plt.title(f"A* TSP Step {step_num}: {title}")
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.grid(True)
        plt.xlim(-1, 5)
        plt.ylim(-1, 5)
        # Lưu tạm vào frames
        png_path = os.path.join(self.frames_dir, f'step_{step_num:03d}.png')
        plt.savefig(png_path, dpi=100, bbox_inches='tight')
        plt.close()
        print(f"Đã lưu step {step_num} (tạm): {png_path}")

    def _calc_f(self, state: Tuple) -> float:
        return 0.0

    def create_gif(self, num_steps: int, duration: float = 8.5):
        """Tạo GIF từ frames_dir và lưu vào results/"""
        if not os.path.exists(self.results_dir):
            os.makedirs(self.results_dir)
        gif_path = os.path.join(self.results_dir, 'a_star_tsp.gif')
        with imageio.get_writer(gif_path, mode='I', duration=duration) as writer:
            for s in range(num_steps + 1):
                frame_path = os.path.join(self.frames_dir, f'step_{s:03d}.png')
                if os.path.exists(frame_path):
                    image = imageio.imread(frame_path)
                    writer.append_data(image)
        print(f"GIF đã được tạo: {gif_path}")

    def plot_final_result(self, path: List[int], cost: float):
        """Vẽ và lưu đồ thị kết quả cuối vào results/"""
        if not os.path.exists(self.results_dir):
            os.makedirs(self.results_dir)
        plt.figure(figsize=(10, 7))
        for i, point in enumerate(self.points):
            plt.scatter(point.x, point.y, c='red', s=1000)
            plt.text(point.x, point.y, str(i), fontsize=14, ha='center', fontweight='bold')
        for i in range(len(path)-1):
            p1 = self.points[path[i]]
            p2 = self.points[path[i+1]]
            plt.arrow(p1.x, p1.y, p2.x - p1.x, p2.y - p1.y, head_width=0.15, head_length=0.15, fc='green', ec='green', linewidth=2)
        plt.text(0.5, -0.1, f"Tổng chi phí: {round(cost, 2)}", transform=plt.gca().transAxes,
                 ha='center', fontsize=16, fontweight='bold', bbox=dict(boxstyle='round', facecolor='lightgreen'))
        plt.title(f"Kết Quả Cuối Cùng: Path Tối Ưu {path} (A* TSP)", fontsize=16, fontweight='bold')
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.grid(True)
        plt.xlim(-1, 5)
        plt.ylim(-1, 5)
        final_path = os.path.join(self.results_dir, 'final_result.png')
        plt.savefig(final_path, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Đồ thị kết quả cuối cùng đã được lưu: {final_path}")

    def cleanup(self):
        """Dọn dẹp frames tạm"""
        if os.path.exists(self.frames_dir):
            shutil.rmtree(self.frames_dir)
        print("Đã dọn dẹp frames tạm.")

Writing visualizer.py


In [5]:
%%writefile runner.py
import os
import json
from typing import List, Tuple
from points import Point, generate_random_points
from graph import TSPGraph
from solver import AStarSolver
from visualizer import Visualizer

class TSPRunner:
    """
    Class quản lý toàn bộ flow TSP: Hỏi lựa chọn load/random, setup points, Solve A*, Visualize theo OOP.
    - Luôn hỏi đầu tiên: load file hay tạo random.
    - Nếu load: Hỏi file_path, tự đọc và set n.
    - Nếu random: Hỏi n, generate random points, lưu JSON.
    """

    def __init__(self):
        self.points: List[Point] = []
        self.n: int = 0
        self.dist_matrix: List[List[float]] = []
        self.graph = None
        self.solver = None
        self.visualizer = None
        self.results_dir = 'results'

        self._interactive_choice()

        from points import get_distance_matrix
        self.dist_matrix = get_distance_matrix(self.points)

    def _interactive_choice(self) -> None:
        """Hỏi lựa chọn: load file hay tạo random."""
        choice = input("Load từ file JSON hay tạo điểm ngẫu nhiên? (load/random, mặc định random): ").strip().lower()
        if choice == 'load':
            self._load_from_file_interactive()
        else:
            self._generate_random_interactive()

    def _load_from_file_interactive(self) -> None:
        """Hỏi file_path, load points, tự set n."""
        file_path = input("Nhập đường dẫn file JSON (ví dụ: /content/random_points_6.json): ").strip()
        if not os.path.exists(file_path):
            print("File không tồn tại. Fallback về tạo ngẫu nhiên.")
            self._generate_random_interactive()
            return

        try:
            with open(file_path, 'r') as f:
                points_data = json.load(f)
            self.points = []
            for data in points_data:
                index = data.get('index', len(self.points))
                x = data.get('x', 0.0)
                y = data.get('y', 0.0)
                self.points.append(Point(x, y, index))
            self.n = len(self.points)
            print(f"Đã load {self.n} điểm từ file: {self.points}")
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Lỗi đọc file: {e}. Fallback về tạo ngẫu nhiên.")
            self._generate_random_interactive()

    def _generate_random_interactive(self) -> None:
        """Hỏi n, generate random points, lưu JSON."""
        try:
            n_input = input("Nhập số lượng điểm (n, mặc định 5, max 12): ").strip()
            self.n = int(n_input) if n_input else 5
            if self.n > 12:
                print("Cảnh báo: n > 12 có thể chậm. Đặt n=12.")
                self.n = 12
            if self.n < 3:
                self.n = 3
        except ValueError:
            self.n = 5
            print("Input không hợp lệ, dùng n=5 mặc định.")

        self.points = generate_random_points(self.n)

        # Tự động lưu JSON
        os.makedirs(self.results_dir, exist_ok=True)
        points_data = [{'index': p.index, 'x': p.x, 'y': p.y} for p in self.points]
        points_file = os.path.join(self.results_dir, f'random_points_{self.n}.json')
        with open(points_file, 'w') as f:
            json.dump(points_data, f, indent=4)
        print(f"Đã lưu điểm ngẫu nhiên vào: {points_file}")

    def _initialize_components(self) -> None:
        """Khởi tạo graph, solver, visualizer (sử dụng self.dist_matrix nếu cần)."""
        self.graph = TSPGraph(self.points)
        self.solver = AStarSolver(self.graph)
        self.visualizer = Visualizer(self.points)
        print("Khởi tạo components (graph, solver, visualizer)...")

    def run(self, max_steps: int = 50) -> Tuple[List[int], float]:
        """Chạy toàn bộ: Solve A*, visualize steps, tạo GIF/final plot, cleanup."""
        self._initialize_components()

        path, cost, steps = self.solver.solve_with_visual(max_steps=max_steps)
        print("\nPath tối ưu:", path)
        print("Tổng chi phí:", round(cost, 2))

        if path:
            # Visualize steps
            for step_num, (current_path, queue_sample) in enumerate(steps):
                title = "Initialization" if step_num == 0 else f"Expanded from {current_path[-1]}, Path: {current_path}"
                self.visualizer.plot_step(step_num, current_path, queue_sample, title)

            # Tạo GIF và final plot
            self.visualizer.create_gif(len(steps) - 1)
            self.visualizer.plot_final_result(path, cost)

        # Cleanup
        self.visualizer.cleanup()

        self._list_results()
        return path, cost

    def _list_results(self) -> None:
        """In danh sách file trong results/."""
        if os.path.exists(self.results_dir):
            files = [f for f in os.listdir(self.results_dir) if not f.startswith('.')]
            print(f"\nNội dung thư mục {self.results_dir}:")
            for f in sorted(files):
                print(f"  - {f}")
        else:
            print("Thư mục results không tồn tại.")

Writing runner.py


In [6]:
%%writefile main.py
from runner import TSPRunner

if __name__ == "__main__":
    runner = TSPRunner()
    path, cost = runner.run(max_steps=50)

Load từ file JSON hay tạo điểm ngẫu nhiên? (load/random, mặc định random): random
Nhập số lượng điểm (n, mặc định 5, max 12): 11
Đã tạo 11 điểm ngẫu nhiên: [Point(0: (0.00, 0.00)), Point(1: (0.55, 4.36)), Point(2: (0.93, 2.58)), Point(3: (1.45, 0.52)), Point(4: (1.45, 0.62)), Point(5: (1.59, 3.58)), Point(6: (0.91, 4.07)), Point(7: (0.07, 4.61)), Point(8: (2.97, 2.31)), Point(9: (4.51, 2.85)), Point(10: (0.62, 1.03))]
Đã lưu điểm ngẫu nhiên vào: results/random_points_11.json
Khởi tạo components (graph, solver, visualizer)...
Step 0: Khởi tạo - Current: 0, Path: [0], g: 0.00, h: 6.00, f: 6.00

Step 1: Pop nút - Current: 0, Visited: 1 (điểm: [0]), g: 0.00
  Mở rộng đến 1: g_new=4.40, h_new=2.12, f_new=6.51
  Mở rộng đến 2: g_new=2.75, h_new=5.36, f_new=8.11
  Mở rộng đến 3: g_new=1.54, h_new=0.47, f_new=2.01
  Mở rộng đến 4: g_new=1.58, h_new=0.47, f_new=2.04
  Mở rộng đến 5: g_new=3.91, h_new=3.75, f_new=7.66
  Mở rộng đến 6: g_new=4.17, h_new=2.12, f_new=6.28
  Mở rộng đến 7: g_new=4.6